In [13]:
import pandas as pd
import numpy as np
import random
import math
from tqdm import tqdm

In [14]:
# data = pd.read_csv(open('data/room.csv'))
# data.replace('?', np.nan).fillna(method='pad'))

data = pd.read_csv('../data/Zoo.csv')
feature_columns = [ col for col in data.columns if col not in ['id','target']]
# for col in feature_columns:
#     data[col] = pd.qcut(data[col], q=5, labels=False, duplicates='drop')
print(data[feature_columns].nunique())
# encode labels column to numbers
# y = data['outcome']

hair        2
feathers    2
eggs        2
milk        2
airborne    2
aquatic     2
predator    2
toothed     2
backbone    2
breathes    2
venomous    2
fins        2
legs        6
tail        2
domestic    2
catsize     2
dtype: int64


In [15]:
len(data)

101

In [24]:
# feature_columns = [ col for col in data.columns if col not in ['id','target']]
# # # # feature_columns=['f4','f5','f6','f7','f17','f18']
# for col in feature_columns:
#     data[col] = pd.qcut(data[col], q=2, labels=False, duplicates='drop')

In [16]:
def calculate_equivalence_classes(data, condition_attrs):
    # 取出条件属性的列索引
    # 设置条件属性为索引
    # print(condition_attrs)
    data_indexed = data.set_index(condition_attrs)
    # 按照索引分组并将分组内结果合并为列表
    equivalence_classes = data_indexed.groupby(level=condition_attrs)[data.columns[0]].apply(list).to_dict()
    return equivalence_classes

In [24]:
def fitness_function(condition_attrs, data, decision_attr):
    # 计算D的信息熵
    total_records = len(data)
    decision_counts = data[decision_attr].value_counts()
    decision_prob = decision_counts / total_records
    entropy_D = -(decision_prob * np.log2(decision_prob)).sum()

    equivalence_classes = calculate_equivalence_classes(data, condition_attrs)
    # print(len(equivalence_classes))
    equivalence_classes_dec = calculate_equivalence_classes(data, decision_attr)

    rule_confidences = []
    entropy_DB = 0
    entropy_DB_sum=0
    total_rule=0

    for equivalence_class in equivalence_classes.values():
        class_size = len(equivalence_class)
        class_prob = class_size / total_records
        for key_decision in equivalence_classes_dec.keys():
            x=set(equivalence_class).intersection(equivalence_classes_dec.get(key_decision))
            # print(len(x))
            if(len(x)>total_records*0.01):
                total_rule+=len(x)
                y = 1-len(x) / class_size
                rule_confidences.append(y)
            if (len(x)>0):
                decision_prob = len(x) / class_size
                entropy_DB =-class_prob * decision_prob * math.log(decision_prob, 2)
                entropy_DB_sum=entropy_DB_sum+entropy_DB

    rule_confidences = np.array(rule_confidences)
    count = len(rule_confidences)

    print('--------------')
    print(count)
    print(total_rule)
    print(entropy_D-entropy_DB_sum)
    print(np.sum(rule_confidences) / count)
    if count == 0:
        return 999;
    if total_rule/total_records<0.4:
        return 999

    print('--------------')
    print(count)
    print(total_rule)
    print(entropy_D-entropy_DB)
    print(np.sum(rule_confidences) / count)
    avg_confidence = np.sum(rule_confidences) / count +0.1*(entropy_D-entropy_DB)

    return avg_confidence

In [26]:
feature_columns=['milk', 'aquatic', 'backbone', 'fins', 'legs']
feature_columns1=['feathers', 'milk', 'backbone', 'fins', 'domestic', 'catsize']
fitness_function(feature_columns1,data,'target')

--------------
14
96
2.152299256249955
0.15178571428571427
--------------
14
96
2.390559682294039
0.15178571428571427


0.3908416825151182

In [28]:
from sklearn.model_selection import StratifiedKFold
from sklearn import svm
from sklearn.preprocessing import StandardScaler


def SVM(trainX,trainY,testX,testY):
    scaler = StandardScaler()
    scaler.fit(trainX)
    trainX3 = scaler.transform(trainX)
    testX3 = scaler.transform(testX)
    clf = svm.SVC(C=10, gamma=0.1, max_iter=200)
    clf.fit(trainX3, trainY)
    test_result = clf.predict(testX3)
    precision = np.sum(test_result == testY)/testY.shape[0]
    return precision

df = pd.read_csv('../data/Soybean/soybean.csv')
feature_columns=['f5', 'f6', 'f9', 'f12', 'f15', 'f16', 'f17', 'f18', 'f19', 'f23', 'f26', 'f27', 'f30', 'f33']
Y = df["target"]
X = df[feature_columns]
# Use stratified k-fold cross-validation
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = []
for train_index, test_index in skf.split(X, Y):
    trainX, testX = X.iloc[train_index], X.iloc[test_index]
    trainY, testY = Y.iloc[train_index], Y.iloc[test_index]
    score = SVM(trainX, trainY, testX, testY)
    scores.append(score)
avg_score = np.mean(scores)
print(avg_score)

0.8960784313725491


C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\model_selection\_split.py:680: UserWarning: The least populated class in y has only 8 members, which is less than n_splits=10.
  UserWarning,
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\svm\_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\svm\_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\svm\_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,


In [29]:
df_train = pd.read_csv('../data/Soybean/soybean-large_train_fillna.csv')
df_test=pd.read_csv('../data/Soybean/soybean-large_test_fillna.csv')
    # encode labels column to numbers
df=pd.concat([df_train,df_test],axis=0)
df.drop(columns='id',inplace=True)
n = len(df)+1
nlist = range(1,n)
df['id'] = nlist
d = df.pop('id')
df.insert(0,'id', d)
df.to_csv('../data/Soybean/soybean.csv',index=False)

In [30]:
df


id                 target   f2   f3   f4   f5   f6   f7   f8   f9  ...  \
0      1  diaporthe-stem-canker  6.0  0.0  2.0  1.0  0.0  1.0  1.0  1.0  ...   
1      2  diaporthe-stem-canker  4.0  0.0  2.0  1.0  0.0  2.0  0.0  2.0  ...   
2      3  diaporthe-stem-canker  3.0  0.0  2.0  1.0  0.0  1.0  0.0  2.0  ...   
3      4  diaporthe-stem-canker  3.0  0.0  2.0  1.0  0.0  1.0  0.0  2.0  ...   
4      5  diaporthe-stem-canker  6.0  0.0  2.0  1.0  0.0  2.0  0.0  1.0  ...   
..   ...                    ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
371  679           2-4-d-injury  0.0  0.0  2.0  1.0  0.0  2.0  2.0  1.0  ...   
372  680       herbicide-injury  0.0  1.0  2.0  0.0  0.0  0.0  0.0  1.0  ...   
373  681       herbicide-injury  2.0  1.0  2.0  0.0  0.0  0.0  0.0  1.0  ...   
374  682       herbicide-injury  0.0  1.0  2.0  0.0  0.0  1.0  3.0  1.0  ...   
375  683       herbicide-injury  2.0  1.0  2.0  0.0  0.0  1.0  3.0  1.0  ...   

     f27  f28  f29  f30  f31  f32  f33  f34  f35  f36  
0    0.0  0.0  0.0  4.0  0.0  0.0  0.0  0.0  0.0  0.0  
1    0.0  0.0  0.0  4.0  0.0  0.0  0.0  0.0  0.0  0.0  
2    0.0  0.0  0.0  4.0  0.0  0.0  0.0  0.0  0.0  0.0  
3    0.0  0.0  0.0  4.0  0.0  0.0  0.0  0.0  0.0  0.0  
4    0.0  0.0  0.0  4.0  0.0  0.0  0.0  0.0  0.0  0.0  
..   ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  
371  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
372  0.0  0.0  3.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  
373  0.0  0.0  3.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  
374  0.0  0.0  3.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  
375  0.0  0.0  3.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  

[683 rows x 37 columns]